## Test
> This test uses the metrics data, created by the [Generator function](https://github.com/mlrun/demo-network-operations/blob/master/notebooks/generator.ipynb) from MLRun's [Network Operations Demo](https://github.com/mlrun/demo-network-operations)  
To test it yourself, please generate this dataset or use any of your available csv/parquet datasets.

In [1]:
from mlrun import code_to_function, mount_v3io, NewTask, mlconf, run_local
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

In [83]:
metrics_path = '/User/demo-network-operations/data/metrics.pq'

### Local Test
Define the aggregate test task

In [87]:
aggregate_task = NewTask(name='aggregate',
                         project='network-operations',
                         params={'metrics': ['cpu_utilization'],
                                 'labels': ['is_error'],
                                 'metric_aggs': ['mean', 'sum'],
                                 'label_aggs': ['max'],
                                 'suffix': 'daily',
                                 'inplace': False,
                                 'window': 5,
                                 'center': True,
                                 'save_to': 'aggregate.pq',
                                 'files_to_select': 2},
                         inputs={'df_artifact': metrics_path},
                         handler=aggregate)

In [90]:
aggregate_run = run_local(aggregate_task)

[mlrun] 2020-07-01 12:39:48,002 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-07-01 12:39:48,010 starting run aggregate uid=569432f3f2314d6b8c746aa05b31fcc0  -> http://mlrun-api:8080
[mlrun] 2020-07-01 12:39:48,070 Aggregating /User/demo-network-operations/data/metrics.pq
[mlrun] 2020-07-01 12:39:48,104 Logging artifact



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
network-operations,...5b31fcc0,0,Jul 01 12:39:48,completed,aggregate,v3io_user=adminkind=handlerowner=adminhost=jupyter-558bf7fbc8-kt6x9,df_artifact,"metrics=['cpu_utilization']labels=['is_error']metric_aggs=['mean', 'sum']label_aggs=['max']suffix=dailyinplace=Falsewindow=5center=Truesave_to=aggregate.pqfiles_to_select=2",,aggregate


to track results use .show() or .logs() or in CLI: 
!mlrun get run 569432f3f2314d6b8c746aa05b31fcc0 --project network-operations , !mlrun logs 569432f3f2314d6b8c746aa05b31fcc0 --project network-operations
[mlrun] 2020-07-01 12:39:48,244 run executed, status=completed


### Test on cluster

Convert the code to an MLRun function

In [92]:
fn = code_to_function('aggregate', handler='aggregate', code_output='function.py')
fn.spec.description = "Rolling aggregation over Metrics and Lables according to specifications"
fn.metadata.categories =  ["data-prep"]
fn.metadata.labels = {'author': 'orz'}
fn.export('function.yaml')

[mlrun] 2020-07-01 12:41:22,171 function spec saved to path: function.yaml


In [ ]:
aggregate_run = fn.apply(mount_v3io()).run(aggregate_task, artifact_path=os.path.abspath('./'))

### Show results

In [91]:
pd.read_parquet(aggregate_run.artifact('aggregate')['target_path'])

cpu_utilization  \
timestamp               company                     data_center      device                           
2020-06-30 06:42:12.454 Wilson_LLC                  Obrien_Mountain  0966571261270        80.900460   
                                                                     8069812479542        82.345824   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054        85.551250   
                                                                     4285071567351        68.295606   
                                                    Dominique_Branch 4579248894449        78.399353   
...                                                                                             ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Zachary_Drives   6001003522699       100.000000   
                                                    Obrien_Mountain  0966571261270        62.157867   
                                                                     8069812479542        59.574487   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054       100.000000   
                                                                     4285071567351       100.000000   

                                                                                       latency  \
timestamp               company                     data_center      device                      
2020-06-30 06:42:12.454 Wilson_LLC                  Obrien_Mountain  0966571261270    3.696670   
                                                                     8069812479542    7.308960   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054    0.000000   
                                                                     4285071567351    7.528826   
                                                    Dominique_Branch 4579248894449    0.000000   
...                                                                                        ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Zachary_Drives   6001003522699  100.000000   
                                                    Obrien_Mountain  0966571261270    1.911734   
                                                                     8069812479542    0.000000   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054  100.000000   
                                                                     4285071567351  100.000000   

                                                                                    packet_loss  \
timestamp               company                     data_center      device                       
2020-06-30 06:42:12.454 Wilson_LLC                  Obrien_Mountain  0966571261270     0.000000   
                                                                     8069812479542     2.567228   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054     0.000000   
                                                                     4285071567351     0.000000   
                                                    Dominique_Branch 4579248894449     0.000000   
...                                                                                         ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Zachary_Drives   6001003522699    50.000000   
                                                    Obrien_Mountain  0966571261270     1.253481   
                                                                     8069812479542     0.000000   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054    50.000000   
                                                                     4285071567351    50.000000   

                                                                                    throughput  \
timestamp               company                     data_center      device        